In [9]:
# ------------------  set up logging ----------------------
import logging
from auditory_cortex.utils import set_up_logging
set_up_logging('info')

import os
import scipy
import numpy as np
import seaborn as sns
from functools import reduce

from auditory_cortex.plotters import tikzplots
from auditory_cortex.analyses import Correlations, STRFCorrelations
from auditory_cortex.plotters.correlation_plotter import RegPlotter
from auditory_cortex.plotters.plotter_utils import PlotterUtils
from statsmodels.stats.multitest import multipletests

import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
%matplotlib inline


### Median correlations for all models

#### timit

In [6]:
threshold = 0.5
model_names = [
    'whisper_tiny', 'whisper_base', 'wav2vec2',
    'wav2letter_modified', 'speech2text', 'deepspeech2',
    ]
identifier = 'timit_trf_lags300_bw50_naplib_matched'
area = 'all'
bin_width = 50
delay = 0
mVocs = True if 'mVocs' in identifier else False
normalized=True
for model_name in model_names:
    corr_obj = Correlations(model_name+'_'+identifier)
    data_dist = corr_obj.get_corr_all_layers_for_bin_width(
                    neural_area=area, bin_width=bin_width, delay=delay,
                    threshold=threshold, normalized=normalized,
                    mVocs=mVocs, 
                )
    print(f"#######################################################")
    print(f"model: {model_name}")
    for layer_id, dist in data_dist.items():
        print(f"layer: {layer_id}, median: {np.median(dist)}")

INFO:Extracting column: normalized_test_cc
INFO:Filtering 'normalizer' using multiple of 0.500 with std dev ...
#######################################################
model: whisper_tiny
layer: 0, median: 0.5196238624345509
layer: 1, median: 0.5793349635051107
layer: 2, median: 0.560012065589323
layer: 3, median: 0.5072174775858294
layer: 4, median: 0.46440022993088004
layer: 5, median: 0.4063860800522521
INFO:Extracting column: normalized_test_cc
INFO:Filtering 'normalizer' using multiple of 0.500 with std dev ...
#######################################################
model: whisper_base
layer: 0, median: 0.5280932295363825
layer: 1, median: 0.5720958721000792
layer: 2, median: 0.5572772551446759
layer: 3, median: 0.5288996030587831
layer: 4, median: 0.4845301305037797
layer: 5, median: 0.46676533849929974
layer: 6, median: 0.4232206829260936
layer: 7, median: 0.41125479767360085
INFO:Extracting column: normalized_test_cc
INFO:Filtering 'normalizer' using multiple of 0.500 with std 

#### mVocs

In [7]:
threshold = 0.5
model_names = [
    'whisper_tiny', 'whisper_base', 'wav2vec2',
    'wav2letter_modified', 'speech2text', 'deepspeech2',
    ]
identifier = 'mVocs_trf_lags300_bw50_naplib_matched'
area = 'all'
mVocs = True if 'mVocs' in identifier else False
bin_width = 50
delay = 0
normalized=True
for model_name in model_names:
    corr_obj = Correlations(model_name+'_'+identifier)
    data_dist = corr_obj.get_corr_all_layers_for_bin_width(
                    neural_area=area, bin_width=bin_width, delay=delay,
                    threshold=threshold, normalized=normalized,
                    mVocs=mVocs, 
                )
    print(f"#######################################################")
    print(f"model: {model_name}")
    for layer_id, dist in data_dist.items():
        print(f"layer: {layer_id}, median: {np.median(dist)}")

INFO:Extracting column: mVocs_normalized_test_cc
INFO:Filtering 'mVocs_normalizer' using multiple of 0.500 with std dev ...
#######################################################
model: whisper_tiny
layer: 0, median: 0.6390401100619133
layer: 1, median: 0.6755318734741179
layer: 2, median: 0.6507962350541424
layer: 3, median: 0.6139949498999512
layer: 4, median: 0.6158836390764476
layer: 5, median: 0.5971326356247014
INFO:Extracting column: mVocs_normalized_test_cc
INFO:Filtering 'mVocs_normalizer' using multiple of 0.500 with std dev ...
#######################################################
model: whisper_base
layer: 0, median: 0.6402062204832104
layer: 1, median: 0.6671388117410714
layer: 2, median: 0.6586770431891543
layer: 3, median: 0.6511502190869117
layer: 4, median: 0.638228035390166
layer: 5, median: 0.6304775504643122
layer: 6, median: 0.6224308497608303
layer: 7, median: 0.6058217705506671
INFO:Extracting column: mVocs_normalized_test_cc
INFO:Filtering 'mVocs_normalizer' 

##

### Tuned & highly-tuned

In [3]:
from scipy import stats
from scipy.stats import mannwhitneyu
from auditory_cortex.neural_data import NormalizerCalculator
from auditory_cortex.neural_data import create_neural_metadata, create_neural_dataset

In [4]:
dataset_name = 'ucsf'
metadata = create_neural_metadata(dataset_name)
sessions = metadata.get_all_available_sessions()
norm_obj = NormalizerCalculator(dataset_name)

stimuli = ['mVocs', 'timit']
stim_wise_tuned_sess_channels = {s:{} for s in stimuli}
stim_wise_session_null_dist = {s:{} for s in stimuli}
stim_wise_session_norm_dist = {s:{} for s in stimuli}

In [5]:
bin_width = 50
p_value = 0.05
stim_wise_p_values = {}
for mVocs in [True, False]:
    stim = 'mVocs' if mVocs else 'timit'
    tuned_channels = []
    all_p_values = []
    highly_tuned_channels = []
    all_p_values = []
    for session in sessions:
        shifted_null = norm_obj.get_normalizer_null_dist_using_random_shifts(
            session, bin_width=bin_width, mVocs=mVocs, force_redo=False
        )
        norm_dist = norm_obj.get_normalizer_for_session(
            session, bin_width=bin_width, mVocs=mVocs
        ) 
        stim_wise_session_null_dist[stim][session] = shifted_null
        stim_wise_session_norm_dist[stim][session] = norm_dist
    
        # shifted_null = stim_wise_session_null_dist[stim][session]
        # norm_dist = stim_wise_session_norm_dist[stim][session]
        tuned_channels = []
        for ch in range(shifted_null.shape[1]):
            dist1 = norm_dist[:,ch]
            dist2 = shifted_null[:,ch]
            stat, p = mannwhitneyu(dist1, dist2, alternative='greater')  # dist1 > dist2
            all_p_values.append(p)
            if p < p_value:
                tuned_channels.append(ch)
        stim_wise_tuned_sess_channels[stim][session] = tuned_channels
    all_p_values = np.array(all_p_values) 
    stim_wise_p_values[stim] = all_p_values

INFO:Getting normalizer dist. for sess-180413, bw-50, mVocs=True


INFO:Getting normalizer dist. for sess-180420, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180501, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180502, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180613, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180622, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180627, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180717, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180719, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180720, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180724, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180728, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180730, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180731, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180807, bw-50, mVocs=True
INFO:Getting normalizer dist. for sess-180808, bw-50, mVocs=True
INFO:Getting normalizer d

In [25]:
alpha_values = [0.05, 0.01, 0.001, 0.0001]

print(f"\t\t alpha:     w/o correction      using Holm-bonferroni correction")
for stim in stimuli:
    print(f"stim: {stim}")
    all_p_values = stim_wise_p_values[stim]
    _, corrected_values, _, _ = multipletests(all_p_values, alpha=p, method='holm')
    for alpha in alpha_values:
        print(f"\t\t{alpha:.4f}: ", end='\t')
        tuned_channels = np.sum(all_p_values < alpha)
        print(f"{tuned_channels}", end='\t\t')
        tuned_with_correction = np.sum(corrected_values < alpha)
        print(f"{tuned_with_correction}")



		 alpha:     w/o correction      using Holm-bonferroni correction
stim: mVocs
		0.0500: 	1231		1190
		0.0100: 	1215		1178
		0.0010: 	1198		1162
		0.0001: 	1190		1152
stim: timit
		0.0500: 	1195		1169
		0.0100: 	1186		1166
		0.0010: 	1176		1161
		0.0001: 	1170		1157


#### number of tuned channels

In [11]:
stim_wise_sess_ch_lists = {stim:{} for stim in stimuli}
total_tuned = {stim:0 for stim in stimuli}
for stim in stimuli:
    neural_areas = ['core', 'non-primary']
    sessions = metadata.get_all_available_sessions()
    stim_wise_sess_ch_lists[stim] = {area:[] for area in neural_areas}
    
    print(f'Stimulus: {stim}')
    for area in neural_areas:
        area_sessions = metadata.get_all_sessions(area)
        # not all sessions are available..
        area_sessions = sessions[np.isin(sessions, area_sessions)]
        area_tuned = 0
        for sess in area_sessions:
            num_channels = len(stim_wise_tuned_sess_channels[stim][sess])
            area_tuned += num_channels
            if num_channels == 0:
                continue
            sess_ch_list = [f'{sess}_{ch}' for ch in stim_wise_tuned_sess_channels[stim][sess]]
            stim_wise_sess_ch_lists[stim][area].extend(sess_ch_list)
        total_tuned[stim] += area_tuned
        print(f'Total tuned channels in {area}: {area_tuned}')
    print(f'Total tuned channels in all areas: {total_tuned[stim]}')

Stimulus: mVocs
Total tuned channels in core: 701
Total tuned channels in non-primary: 530
Total tuned channels in all areas: 1231
Stimulus: timit
Total tuned channels in core: 707
Total tuned channels in non-primary: 488
Total tuned channels in all areas: 1195


In [12]:
for area in ['core', 'non-primary']:
    print(f'Area: {area}')
    list_tuned = {}
    for stim in ['timit', 'mVocs']:
        list_tuned[stim] = stim_wise_sess_ch_lists[stim][area]
        print(f"\t Tuned for {stim}: {len(list_tuned[stim])}")
    tuned_both = set(list_tuned['mVocs']).intersection(set(list_tuned['timit']))
    print(f"\t Tuned for both: {len(tuned_both)}")
    print('---')

Area: core
	 Tuned for timit: 707
	 Tuned for mVocs: 701
	 Tuned for both: 580
---
Area: non-primary
	 Tuned for timit: 488
	 Tuned for mVocs: 530
	 Tuned for both: 341
---


#### number of highly-tuned

In [13]:
gap = 0.5
stim_wise_highly_tuned = {stim:{} for stim in stimuli}
for stim in stimuli:
    for session in sessions:
        norm_dist = stim_wise_session_norm_dist[stim][session]
        shifted_null = stim_wise_session_null_dist[stim][session]

        norm_means = np.mean(norm_dist, axis=0)
        null_means = np.mean(shifted_null, axis=0)

        null_std = np.std(shifted_null, axis=0)
        thresh = null_means + (gap * null_std)
        highly_tuned_chs = np.where(norm_means > thresh)[0]
        stim_wise_highly_tuned[stim][session] = highly_tuned_chs

In [14]:
print(f"Gap: {gap}")
stim_wise_sess_ch_lists = {stim:{} for stim in stimuli}
total_highly_tuned = {stim:0 for stim in stimuli}
for stim in ['timit', 'mVocs']:
    neural_areas = ['core', 'non-primary']
    sessions = metadata.get_all_available_sessions()
    stim_wise_sess_ch_lists[stim] = {area:[] for area in neural_areas}
    
    print(f'Stimulus: {stim}')
    for area in neural_areas:
        area_sessions = metadata.get_all_sessions(area)
        # not all sessions are available..
        area_sessions = sessions[np.isin(sessions, area_sessions)]
        area_tuned = 0
        for sess in area_sessions:
            num_channels = len(stim_wise_highly_tuned[stim][sess])
            area_tuned += num_channels
            if num_channels == 0:
                continue
            sess_ch_list = [f'{sess}_{ch}' for ch in stim_wise_highly_tuned[stim][sess]]
            stim_wise_sess_ch_lists[stim][area].extend(sess_ch_list)
        total_highly_tuned[stim] += area_tuned
        print(f'Total highly tuned channels in {area}: {area_tuned}')
    print(f'Total highly tuned channels in all areas: {total_highly_tuned[stim]}')

Gap: 0.5
Stimulus: timit
Total highly tuned channels in core: 300
Total highly tuned channels in non-primary: 104
Total highly tuned channels in all areas: 404
Stimulus: mVocs
Total highly tuned channels in core: 315
Total highly tuned channels in non-primary: 174
Total highly tuned channels in all areas: 489


In [15]:
print(f"Gap: {gap}")
for area in ['core', 'non-primary']:
    print(f'Area: {area}')
    list_tuned = {}
    for stim in ['timit', 'mVocs']:
        list_tuned[stim] = stim_wise_sess_ch_lists[stim][area]
        print(f"\t Highly tuned for {stim}: {len(list_tuned[stim])}")
    tuned_both = set(list_tuned['mVocs']).intersection(set(list_tuned['timit']))
    print(f"\t Highly tuned for both: {len(tuned_both)}")
    print('---')

Gap: 0.5
Area: core
	 Highly tuned for timit: 300
	 Highly tuned for mVocs: 315
	 Highly tuned for both: 246
---
Area: non-primary
	 Highly tuned for timit: 104
	 Highly tuned for mVocs: 174
	 Highly tuned for both: 78
---
